In [2]:
import pandas as pd
import xlwings as xw
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import os
import sys
sys.path.append('../')
from demo.sim.jet_sim import *
from tqdm import tqdm

In [3]:
def data_nplist(file_path:str = '../data/CFD0.8_0.2.csv',columns_index = 0,is_mean = True,is_reset = True):
    '''
    file_path: CFD数据路径
    columns_index: 读取的列数
    is_mean: 是否取平均值
    is_reset: 是否归零
    用途：读取CFD数据，返回numpy数组
    '''
    pd_data = pd.read_csv(file_path)
    j = pd_data.iloc[0,0]
    num = 0
    cut_list = [0]
    for i in pd_data.iloc[:,0]:
        if abs(j-i) > 1:
            j = i
            cut_list.append(num)
        else:
            j = i
        num += 1
    cut_list.append(num-1)

    res = dict()
    for i in range(len(cut_list)-1):
        res[f'ID{i}'] = pd_data.iloc[cut_list[i]:cut_list[i+1],]

    min_num = 10000
    for i in range(len(res)):
        if len(res[f'ID{i}']) < min_num:
            min_num = len(res[f'ID{i}'])

    np_list = np.empty((min_num,0))
    for i in res.keys():
        b = np.asarray(res[i].iloc[:min_num,columns_index]).reshape(-1,1)
        np_list = np.concatenate((np_list,b),axis=1)
    if is_mean:
        np_list = np_list.mean(axis=1)
    if is_reset:
        np_list = np_list - np_list.min() # 坐标归零
    return(np_list)

def reslut_plot(TIME,DELTA_T,K,U,EXHALE,file_path='../data/CFD0.8_0.2.csv',name = ['sim','cfd']):
    ''' 
    TIME: 模拟时间
    DELTA_T: 模拟时间间隔
    K: 衰减系数
    U: 环境风速
    EXHALE: 呼气速度
    file_path: CFD数据路径
    name: 图例名称
    用途：模拟结果与CFD结果对比
    '''
    sim_data = transform_pd(sim_jet(TIME,DELTA_T,K,U,EXHALE))

    cfd_data_x = data_nplist(file_path,columns_index = 0)
    cfd_data_y = data_nplist(file_path,columns_index = 1)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sim_data['x'], y=sim_data['y'], mode='lines', name=name[0],marker_color='#ef8a62'))
    fig.add_trace(go.Scatter(x=cfd_data_x, y=cfd_data_y, mode='lines', name=name[1],marker_color='#67a9cf'))
    fig.update_xaxes(range=[0, 2])
    fig.update_yaxes(range=[0, 0.3])
    fig.update_layout(title=f'Attenuation coefficient={K},\
                    <br />Ambient airflow={U}m/s, Exhale={EXHALE[1]}m/s', xaxis_title='x(m)', yaxis_title='y(m)',
                    font_family = 'Times New Roman',width=800, height=600)
    return fig

def output(TIME,DELTA_T,U,E,name = ['Calc','Sim']):
    FILE_PATH = f'../data/{E}_{U}.csv'
    return reslut_plot(TIME,DELTA_T,K,U,[0,E],FILE_PATH,name = name)

In [8]:
# 导出CFD数据为csv

file_path = '../data/0.55_0.6.csv'
x = data_nplist(file_path=file_path,columns_index = 0,is_mean = True)
y = data_nplist(file_path=file_path,columns_index = 1,is_mean = True)
ppm = data_nplist(file_path=file_path,columns_index = 3,is_mean = True,is_reset = False)
pd.DataFrame({'x':x,'y':y,'ppm':ppm}).to_csv(f'../output/CFD_{file_path[8:]}',index=False)

In [10]:
TIME = 20
DELTA_T = 0.005
K = 0.5
U = [0.2,0.4,0.6]
EXHALE = [0.55,0.8,1.1]

In [6]:
sim_data = transform_pd(sim_jet(Total_time=TIME,delta_t=DELTA_T,k=K,U=U[0],U_origin=[0,EXHALE[0]]))
fig = px.scatter(sim_data, x="x", y="y",color='ppm',color_continuous_scale='earth',
                 range_x=[0,1],range_y=[0,0.3],width=800, height=600)
fig.show()

In [15]:
transform_pd(sim_jet(Total_time=TIME,delta_t=DELTA_T,k=K,U=U[2],U_origin=[0,EXHALE[0]])).to_csv(f'../output/SIM_{EXHALE[0]}_{U[2]}.csv',index=False)